### Anaconda Environment

In [ ]:
# Create
## conda create -n {desired name} python = {desired version}
## conda activate {desired name}

# Activate, Deactivate
## source activate {desired env name}
## source deactivate

# Remove
## conda remove -n {desired env name} --all

### PyTorch

- GPU와 CPU를 사용
- 프로그래밍 언어 Lua로 개발
- PyTorch로 넘어오면서 자동 미분 모듈. 따로 backward 함수를 구현하지 않아도 됨

Tensor

- Very similar to numpy ndarrays (tensor는 GPU에서도 사용)

In [8]:
import torch
import numpy as np

#### Tensors: creation

In [9]:
data = [[1, 2], [3, 4]]
x_data = torch.tensor(data) # 2*2 tensor

In [10]:
x_data = torch.tensor(3.14159) # scalar tensor

In [11]:
x_data = torch.tensor([]) # empty tensor

In [12]:
# From numpy arrays
np_array = np.array(data)
x_np = torch.from_numpy(np_array)

In [14]:
# Tensor creation using APIs
shape = (2, 3,)
torch.rand(shape)

tensor([[0.5275, 0.4973, 0.3371],
        [0.8353, 0.3954, 0.4422]])

In [15]:
torch.ones(shape)

tensor([[1., 1., 1.],
        [1., 1., 1.]])

In [16]:
torch.zeros(shape)

tensor([[0., 0., 0.],
        [0., 0., 0.]])

In [17]:
# Tensor creation from other tensors
x_ones = torch.ones_like(x_data) # retains the properties of x_data
x_ones = torch.rand_like(x_data, dtype=torch.float) # overrides the datatype of x_data

#### Tensors: attributes and GPU

In [24]:
# Tensor attributes
tensor = torch.rand(3, 4)
tensor.shape

torch.Size([3, 4])

In [25]:
tensor.dtype

torch.float32

In [26]:
tensor.device

device(type='cpu')

In [ ]:
# Run tensor on the GPU
tensor = tensor.to('cuda')
tensor.device

#### Tensor operations

In [28]:
tensor = torch.ones(4, 4)
tensor[:, 1] = 0 # indexing
tensor

tensor([[1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.]])

In [29]:
torch.cat([tensor, tensor, tensor], dim=1) # join

tensor([[1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1.],
        [1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1.],
        [1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1.],
        [1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1.]])

In [30]:
tensor + tensor * tensor - tensor # element-wise arithmetic opearations

tensor([[1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.]])

In [31]:
tensor @ tensor # matrix multiplication

tensor([[3., 0., 3., 3.],
        [3., 0., 3., 3.],
        [3., 0., 3., 3.],
        [3., 0., 3., 3.]])

### Load Data

- Datasets and DataLoader
    - All datasets are subclass of torch.utils.data.Dataset
        - You can make your own datasets by inheriting Dataset class
        - Some famous datasets is provided in torchvision.datasets
    - DataLoader reads datasets and handless multi-process mini-batch data loading

- one epoch = one forward pass and one backward pass of all the training examples
- batch size = the number of traning examples in one forward/backward pass. The higher the batch size, the more memory space you'll need
- number of iterations = number of passes, each pass using [batch size] number of examples. To be clear, one pass = one forward pass + one backward pass (we do not count the forward pass and backward pass as two different passes)

In [ ]:
# Training cycle
for epoch in range(training_epochs):
    # Loop over all batches
    for i in range(total_batch):

#### Datasets from torchvision

In [32]:
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt

In [35]:
training_data = datasets.FashionMNIST(
root='data',
train=True,
download=True,
transform=ToTensor())

test_data = datasets.FashionMNIST(
root='data',
train=False,
download=True,
transform=ToTensor())

#### Custom dataset

- Methods you should define when inherit from torch.utils.Dataset
- __init__(self,[]) : 데이터셋 루트 디렉토리 / transform / 이미지 path ...
- __len__(self) : 데이터셋의 length
- __getitem__(self, idx) : (idx)th data 를 return. (DataLoader call this method)

In [ ]:
class DiabetesDataset(Dataset):
    """Diabetes dataset."""
    
    # Initialize your data, download, etc.
    def __init__(self):
        
    def __getitem__(self, index):
        
    def __len__(self):
        return
    
dataset = DiabetesDataset()
train_loader = DataLoader(dataset=dataset,
                          batch_Size=32,
                          shuffle=True,
                          num_workers=2)

#### Dataloader
- Methods reads datasets and handles **multi-process mini-batch data loading**
    - Multi-precess mini-batch loader can avoid bottleneck on reading and transferring data.
    - DataLoader is an iterable that abstracts this complexity for us in an easy API.

In [36]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=True)

#### Build the Neural Network

- Define two method in network class "__init__" and "forward"
- "__init__" : Need to implement the network architecture using torch.nn namespace.
- "forward" : define how do you compute the forward path of the neural network.

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

#### Training

1. Define Model
2. Define Dataloader
3. Define Optimizer
4. Iterate the below lines until reaching the max_epoch.
    - (1) Sample a batch from the data loader
    - (2) Predict the output using the neural network
    - (3) Compute the error between the answer and prediction
    - (4) Compute the gradient using optimizer
    - (5) Backpropagate the gradient to optimize the NN parameters

In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

#### Save and Load Model Weights

torch.save, torch.load

In [ ]:
model = models.vgg16(pretrained=True)
torch.save(model.state_dict(), 'model_weights.pth')

In [ ]:
model = models.vgg16()
model.load_state_dict(torch.load('model_weights.pth'))
model.eval()

#### Transfer learning

- pre-trained model을 사용한 image classification model fine-tuning

In [ ]:
model_ft = model.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features

# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_name))
model_ft.fc = nn.Linear(num_ftrs, 2) # 마지막 FCL를 2개로 바꿈.

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(mode_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)